# Prepare files and install conda

In [ ]:
!wget "https://raw.githubusercontent.com/yukimasano/single-img-extrapolating/master/data_generation/make_single_img_dataset.py"
!wget "https://raw.githubusercontent.com/yukimasano/single-img-extrapolating/master/in1k/requirements.txt"
!wget "https://raw.githubusercontent.com/yukimasano/single-img-extrapolating/master/data_generation/images/animals.png"
!wget "https://raw.githubusercontent.com/yukimasano/single-img-extrapolating/master/data_generation/images/ameyoko.jpg"
!wget "https://raw.githubusercontent.com/yukimasano/single-img-extrapolating/master/data_generation/images/img_a.jpg"
!mkdir train_set
!wget "https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh"

In [ ]:
!bash Miniconda3-latest-Linux-x86_64.sh

In [ ]:
!/root/miniconda3/bin/conda list   # check that conda is installed

# Transform requirements.txt and set up environment

In [4]:
SCRIPT = '''#!/usr/bin/env awk -f
#' Author: Mervin Fansler
#' GitHub: @mfansler
#' License: MIT
#'
#' Basic usage
#' $ conda list --export | awk -f list_export_to_yaml.awk
#'
#' Omitting builds with 'no_builds'
#' $ conda list --export | awk -v no_builds=1 -f list_export_to_yaml.awk
#'
#' Specifying channels with 'channels'
#' $ conda list --export | awk -v channels="conda-forge,defaults" -f list_export_to_yaml.awk

BEGIN {
  FS="=";
  if (channels) split(channels, channels_arr, ",");
  else channels_arr[0]="defaults";
}
{
  # skip header
  if ($1 ~ /^#/) next;

  if ($3 ~ /pypi/) {  # pypi packages
    pip=1;
    pypi[i++]="    - "$1"=="$2" ";
  } else {  # conda packages
    if ($1 ~ /pip/) pip=1;
    else {  # should we keep builds?
      if (no_builds) conda[j++]="  - "$1"="$2" ";
      else conda[j++]="  - "$1"="$2"="$3" ";
    }
  }
}
END {
  # emit channel info
  print "channels: ";
  for (k in channels_arr) print "  - "channels_arr[k]" ";

  # emit conda pkg info
  print "dependencies: ";
  for (j in conda) print conda[j];

  # emit PyPI pkg info
  if (pip) print "  - pip ";
  if (length(pypi) > 0) {
      print "  - pip: ";
      for (i in pypi) print pypi[i];
  }
}
'''

with open('list_export_to_yaml.awk', 'w') as f:
    f.write(SCRIPT)

In [5]:
!awk -f list_export_to_yaml.awk -v channels='anaconda,conda-forge,defaults,pytorch,intel,bioconda' requirements.txt > req.yaml

In [6]:
!/root/miniconda3/bin/conda config --add channels anaconda
!/root/miniconda3/bin/conda config --add channels conda-forge
!/root/miniconda3/bin/conda config --add channels defaults
!/root/miniconda3/bin/conda config --add channels pytorch
!/root/miniconda3/bin/conda config --add channels intel
!/root/miniconda3/bin/conda config --add channels bioconda

!/root/miniconda3/bin/conda config --show channels

channels:
  - bioconda
  - intel
  - pytorch
  - defaults
  - conda-forge
  - anaconda


In [ ]:
!/root/miniconda3/bin/conda env create --name venv --file req.yaml   # takes A LOT OF TIME

In [8]:
!/root/miniconda3/bin/activate venv

# running make_single_img_dataset.py script

In [9]:
# pass your --imgpath below
!/root/miniconda3/envs/venv/bin/python make_single_img_dataset.py --imgpath ameyoko.jpg --targetpath ./train_set --threads 1

Namespace(img_size=32, batch_size=32, num_imgs=50000, threads=1, vflip=False, deg=30, shear=30, cropfirst=True, initcrop=0.5, scale=[500, 1], randinterp=False, debug=False, imgpath='ameyoko.jpg', targetpath='./train_set', img_per_thread=50000)
will save 50000 patches in ./train_set32_singleameyoko_init0.5_deg30_scale500_1_shear30_randinterp_False_vflipFalse_cropfirstTrue_new_50000/train/dummy
/root/miniconda3/envs/venv/lib/python3.9/site-packages/torchvision/transforms/transforms.py:852: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/root/miniconda3/envs/venv/lib/python3.9/site-packages/torchvision/transforms/transforms.py:1361: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(
/root/miniconda3/envs/venv/lib/python3.9/site-packages/torchvision/transforms/transforms.py:1375: UserWarning: Argument fillcolor is depr

In [13]:
# path below also depends on your imgpath
!ls train_set32_singleameyoko_init0.5_deg30_scale500_1_shear30_randinterp_False_vflipFalse_cropfirstTrue_new_50000/train/dummy | wc -l

49984


# dataset archivation

In [14]:
# path below also depends on your imgpath
!mv train_set32_singleameyoko_init0.5_deg30_scale500_1_shear30_randinterp_False_vflipFalse_cropfirstTrue_new_50000/train/dummy single_image_dataset

In [ ]:
!zip -r single_image_dataset.zip single_image_dataset